In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

### Dataset:
Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California,
School of Information and Computer Science.

Donated by:
P. Savicky
Institute of Computer Science, AS of CR
Czech Republic
savicky '@' cs.cas.cz

In [2]:
cols = ["fLength", "fWidth", "fSize", "fConc", "fConcl", "fAsym", "fM3Long" ,"fM3Trans" , "fAlpha", "fDist", "class"]
df = pd.read_csv('magic04.data', names=cols)
df.head()

,fLength,fWidth,fSize,fConc,fConcl,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [3]:
df["class"] = (df["class"] == "g").astype(int) # 1 for g, 0 for h 

In [4]:
df.head()

,fLength,fWidth,fSize,fConc,fConcl,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,1
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,1
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,1
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,1
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,1


In [ ]:
for label in cols[:-1]:
    plt.hist(df[df["class"]==1][label],color='blue',label='gamma',alpha=0.7,density=True)
    plt.hist(df[df["class"]==0][label],color='red',label='hadron',alpha=0.7,density=True)
    plt.title(label)
    plt.ylabel("probability")
    plt.xlabel(label)
    plt.legend()
    plt.show()

# train, validation, test datasets

In [5]:
train ,valid ,test = np.split(df.sample(frac=1),[int(0.6*len(df)), int(0.8*len(df))])  #shuffle and split dataset 

c:\Users\pranjul saxena\ml\aiml-env\Lib\site-packages\numpy\_core\fromnumeric.py:54: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [6]:
def scale_dataset(dataframe ,oversample=False):
    x= dataframe[dataframe.columns[:-1]].values  
    y= dataframe[dataframe.columns[-1]].values
    
    scalar = StandardScaler()
    x= scalar.fit_transform(x)
    
    if oversample:    #optional oversampling to balance classes
        ros = RandomOverSampler()
        x, y = ros.fit_resample(x, y) #oversample minority class
        
    data = np.hstack((x,np.reshape(y,(-1,1))))
    
    return data, x, y

In [7]:
train, x_train, y_train = scale_dataset(train, oversample=True) #oversample training set
valid, x_valid, y_valid = scale_dataset(valid, oversample=False) #do not oversample validation set 
test, x_test, y_test = scale_dataset(test, oversample=False)     #do not oversample test set

# KNN

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [9]:
knn_model = KNeighborsClassifier(n_neighbors=3) #3-NN model
knn_model.fit(x_train, y_train) #train the model

,"n_neighbors n_neighbors: int, default=5Number of neighbors to use by default for :meth:`kneighbors` queries.",3
,"weights weights: {'uniform', 'distance'}, callable or None, default='uniform'Weight function used in prediction. Possible values:- 'uniform' : uniform weights. All points in each neighborhood are weighted equally.- 'distance' : weight points by the inverse of their distance. in this case, closer neighbors of a query point will have a greater influence than neighbors which are further away.- [callable] : a user-defined function which accepts an array of distances, and returns an array of the same shape containing the weights.Refer to the example entitled:ref:`sphx_glr_auto_examples_neighbors_plot_classification.py`showing the impact of the `weights` parameter on the decisionboundary.",'uniform'
,"algorithm algorithm: {'auto', 'ball_tree', 'kd_tree', 'brute'}, default='auto'Algorithm used to compute the nearest neighbors:- 'ball_tree' will use :class:`BallTree`- 'kd_tree' will use :class:`KDTree`- 'brute' will use a brute-force search.- 'auto' will attempt to decide the most appropriate algorithm based on the values passed to :meth:`fit` method.Note: fitting on sparse input will override the setting ofthis parameter, using brute force.",'auto'
,"leaf_size leaf_size: int, default=30Leaf size passed to BallTree or KDTree. This can affect thespeed of the construction and query, as well as the memoryrequired to store the tree. The optimal value depends on thenature of the problem.",30
,"p p: float, default=2Power parameter for the Minkowski metric. When p = 1, this is equivalentto using manhattan_distance (l1), and euclidean_distance (l2) for p = 2.For arbitrary p, minkowski_distance (l_p) is used. This parameter is expectedto be positive.",2
,"metric metric: str or callable, default='minkowski'Metric to use for distance computation. Default is ""minkowski"", whichresults in the standard Euclidean distance when p = 2. See thedocumentation of `scipy.spatial.distance`_ andthe metrics listed in:class:`~sklearn.metrics.pairwise.distance_metrics` for valid metricvalues.If metric is ""precomputed"", X is assumed to be a distance matrix andmust be square during fit. X may be a :term:`sparse graph`, in whichcase only ""nonzero"" elements may be considered neighbors.If metric is a callable function, it takes two arrays representing 1Dvectors as inputs and must return one value indicating the distancebetween those vectors. This works for Scipy's metrics, but is lessefficient than passing the metric name as a string.",'minkowski'
,"metric_params metric_params: dict, default=NoneAdditional keyword arguments for the metric function.",None
,"n_jobs n_jobs: int, default=NoneThe number of parallel jobs to run for neighbors search.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.Doesn't affect :meth:`fit` method.",None


In [10]:
y_pred = knn_model.predict(x_test) #predict on test set

In [11]:
print(classification_report(y_test, y_pred)) #print precision, recall, f1-score

              precision    recall  f1-score   support

           0       0.70      0.72      0.71      1282
           1       0.86      0.84      0.85      2522

    accuracy                           0.80      3804
   macro avg       0.78      0.78      0.78      3804
weighted avg       0.81      0.80      0.80      3804



In [12]:
y_pred

array([1, 1, 1, ..., 0, 1, 1], shape=(3804,))

In [13]:
y_test

array([1, 1, 1, ..., 0, 1, 1], shape=(3804,))

# Naive Bayes

In [25]:
from sklearn.naive_bayes import GaussianNB

In [26]:
nb_model = GaussianNB() #Gaussian Naive Bayes model
nb_model = nb_model.fit(x_train, y_train) #train the model

In [27]:
y_pred = nb_model.predict(x_test) #predict on test set
print(classification_report(y_test, y_pred)) #print precision, recall, f1-score

              precision    recall  f1-score   support

           0       0.64      0.41      0.50      1282
           1       0.75      0.88      0.81      2522

    accuracy                           0.72      3804
   macro avg       0.69      0.65      0.66      3804
weighted avg       0.71      0.72      0.71      3804



In [28]:
from sklearn.linear_model import LogisticRegression

In [29]:
lg_model = LogisticRegression() #Logistic Regression model
lg_model = lg_model.fit(x_train, y_train) #train the model

In [33]:
y_pred = lg_model.predict(x_test) #predict on test set
print(classification_report(y_test, y_pred)) #print precision, recall, f1-score

              precision    recall  f1-score   support

           0       0.66      0.71      0.68      1282
           1       0.84      0.81      0.83      2522

    accuracy                           0.78      3804
   macro avg       0.75      0.76      0.75      3804
weighted avg       0.78      0.78      0.78      3804



In [34]:
from sklearn.svm import SVC

In [35]:
svm_model = SVC() #Support Vector Machine model 
svm_model = svm_model.fit(x_train, y_train) #train the model

In [36]:
y_pred = svm_model.predict(x_test) #predict on test set
print(classification_report(y_test, y_pred)) #print precision, recall, f1-score

              precision    recall  f1-score   support

           0       0.78      0.78      0.78      1282
           1       0.89      0.89      0.89      2522

    accuracy                           0.85      3804
   macro avg       0.83      0.83      0.83      3804
weighted avg       0.85      0.85      0.85      3804

